# Alone in the dark

First let's see the problem

In [ ]:
#!/usr/bin/env python

import gmpy2
from hashlib import sha256
from secret import u, v, x, y

assert ((u+1)**2 + u**2 - v**2)**2 + ((x+1)**3 - x**3 - y**2)**4 + (gmpy2.is_prime(v) + 1)**6 + (gmpy2.is_prime(y) - 1)**8 + (len(bin(u)[2:]) - 664)**10 + (len(bin(x)[2:]) - 600)**12 == 664 - 600

flag = 'CCTF{' + sha256(str(u) + str(v) + str(x) + str(y)).hexdigest() + '}'

It's a single assert, asking that a sum of even powers is equal to $64$. In particular, all the addends will be positive.

The first thing to notice is that the last two powers ($10$ and $12$) are too big, hence the argument must be $0$ or $\pm1$; this means that $u$ has $664\pm1$ digits in base $2$ and $x$ has $600\pm1$ digits. This will be helpful later.

Then we have $({\tt is\_prime}(v)+1)^6+({\tt is\_prime}(y)-1)^8$: if $v$ is prime, then all the other powers must be zero!
Let's see if there exists a solution with $v$ not prime.

Then we have something of the form $a^2+b^4=64-1-\varepsilon_1-\varepsilon_2-\varepsilon_3$, where the $\varepsilon_i$ are $0$ or $1$; hence we have the three equations $a^2+b^4=63,62,61,60$ and none of them has an integer solution.

In [1]:
res = [60,61,62,63]
for a in range(8):
    for b in range(3):
        z = a^2+b^4
        if z in res:
            print z        

Ok, so we have $v$ prime and all the other powers equal to zero:
 - $v,y$ primes
 - $(u+1)^2+u^2-v^2=0$ and $(x+1)^3-x^3-y^2=0$
 - $\lfloor\log_2 u\rfloor+1=664$ and $\lfloor\log_2 x\rfloor+1=600$
 
 We will use the middle equations, which are examples of the so-called [Pell's equation](https://en.wikipedia.org/wiki/Pell%27s_equation) since we can rewrite them as
 $$ (2u+1)^2-2v^2=1 $$
 $$ (2y)^2-3(2x+1)^2 = 1 $$
 
 Now the path is straightforward: each Pell has an infinite number of solutions, but parametrized by one integer $n$; so we pick the one solution of the right size, and hope that $v$ and $y$ are actually primes!

### Let's solve $(2u+1)^2-2v^2=1$!

The theory of the Pell's equations says that every solution is a power of the fundamental unit of $\mathbb Q[\sqrt2]$, i.e 
$$ (2u_n+1)+v_n\sqrt2 = (1+\sqrt2)^n $$

Sage is made to do this sort of computations, and we use it!

To find $n$ we use the fact that $2u_n+1$ is VERY near to $(1+\sqrt2)^n$; since we know how big $u$ is, we can find $n$.

In [42]:
K.<a> = QuadraticField(2)
t = K.units()[0]
print t

phi = K.embeddings(RR)[1]
664/log(phi(t),2) #this is the approximate exponent n we are looking for

a + 1


522.195961701310

In [47]:
for n in range(510,530):
    u = ((1+a)^n + (1-a)^n)/2
    v = ((1+a)^n - (1-a)^n)/(2*a)
    if is_prime(Integer(v)):
        break

# transform the solutions to the actual numbers
u = Integer((u-1)/2)
v = Integer(v)
print "u =", u
print "v =", v

u = 38870796548368940451592529482185869982938448205812640195914560739542103841403178847163517462769143179065031576973812014377488506777895445800461891869308645201761858965032907136032847098509289762520539
v = 54971607658948646301386783144964782698772613513307493180078896702918825851648683235325858118170150873214978343601463118106546653220435805362395962991295556488036606954237309847762149971207793263738989


### Let's solve $(2y)^2-3(2x+1)^2=1$!

This is identical to the one above, but we work in the field $\mathbb Q[\sqrt3]$.

The solutions are $(2y_n)+(2x_n+1)\sqrt3=(2+\sqrt3)^n$, and as before we find the proper $n$.

In [48]:
K.<a> = QuadraticField(3)
t = K.units()[0]
t = -1/t
print t

phi = K.embeddings(RR)[1]
600/log(phi(t),2) #this is the approximate exponent n we are looking for

a + 2


315.794688127156

In [50]:
for n in range(310,320):
    y = ((2+a)^n + (2-a)^n)/2
    x = ((2+a)^n - (2-a)^n)/(2*a)
    if is_even(Integer(y)) and is_prime(Integer(y/2)):
        break
        
# transform back
y = Integer(y/2)
x = Integer((x-1)/2)

print "x =", x
print "y =", y

x = 2929219721139577720733862051859801690342725739320715630102152440665051724895027651815801589822478988305846846083549661332897318938724576681923803796059612952236038798314710140134264
y = 5073557383546487137945410473466556718830129339025213837451156233563658135296353459994544781708539660966095175852902937975236171859961262538393568510313468641105066779787434237464141


## The end!

In [52]:
from hashlib import sha256
flag = 'CCTF{' + sha256(str(u) + str(v) + str(x) + str(y)).hexdigest() + '}'
print flag

CCTF{07f594e5fb8f6d5f82e5cce06e2a2c74c1bffce370cd904821fdd71027faa084}
